# Autor
Notebook criado por [Wallacy Pasqualini](https://www.linkedin.com/in/wallacypasqualini/).

Visite meu [GitHub](https://github.com/WallPasq) para ver meus projetos.

# O problema

> 💡 O objetivo desse Notebook está em mostrar o poder do Z-score. Leia até o final para entender o que foi feito em cada célula.

> ❗️ Abaixo conto uma história inspirada em algo que aconteceu comigo, com detalhes e exemplos fictícios para anonimizar devidamente os dados.

Recentemente passei por um desafio onde me foi passado uma meta de Ticket Médio geral, e eu precisava verificar o quanto o Ticket Médio de cada canal variou em relação à meta. 

Parece uma tarefa bem simples a princípio, certo? Só que tive dois problemas logo no início:

1. Os canais estavam segmentados em três colunas, onde cada coluna definia uma especificação diferente. Exemplo:

    | Tipo   | Canal      | Direcionamento   |
    |--------|------------|------------------|
    | Ads    | Facebook   | Blog             | 
    
    <br />
1. O Ticket Médio mudava muito dependendo do canal, o que deixava alguns muito acima da meta e outros muito abaixo. Exemplo:

    | Tipo   | Canal      | Direcionamento   | Meta     | Ticket Médio | Desvio |
    |--------|------------|------------------|----------|--------------|--------|
    | Ads    | Facebook   | Blog             | R$ 6.000 | R$ 10.000    | 67%    |
    | E-mail | Campanha   | Site             | R$ 6.000 | R$ 400       | - 93%  |

    <br />

O que você faria, nesse caso? Entregaria os números dessa forma?

Eu aprendi com o professor Daniel, na [Data Science Academy](https://www.datascienceacademy.com.br/), que o valor de um profissional na área de Dados está na sua capacidade de reconhecer e resolver problemas.

Por isso, para mim se tornou inaceitável entregar os valores dessa forma, pois estavam longe de representar a realidade.

Mas, como ajustar a meta de Ticket Médio para cada canal?

Talvez calcular um fator, que seria o Ticket Médio de cada combinação das três colunas dividido pelo Ticket Médio geral.

Depois bastaria multiplicar esse fator pela Meta, obtendo assim a Meta ajustada.

Essa seria uma possível solução, se não fosse o fato de eu precisar levar esses dados para o Power BI, permitindo filtros de seleção múltipla por cada coluna, que poderiam ser ou não combinados.

E agora? Talvez calcular o desvio no nível de cada coluna e também nos níveis das combinações?

Mas e se o número de colunas aumentasse depois? Certamente essa solução não seria viável nem escalável.

Isso além do fato da tabela ter milhões de linhas, o que prejudicaria a performance no Power BI para calcular o Ticket Médio de cada combinação e dividir pelo Ticket Médio geral, mesmo utilizando o contexto de filtro.

Não consegui encontrar uma solução naquele momento. Contudo, meu amigo [Gabriel Fonseca](https://www.linkedin.com/in/gabriel-felipe-fonseca/) me ensinou o seguinte:

> Às vezes é melhor dormir com a informação e deixar a nossa mente "pensar durante o repouso".

E foi o que eu fiz!

# Propriedades do Z-score

Você já ouviu falar no Z-score? Em uma definição simples, ele é a quantidade de desvios padrão que um valor está acima ou abaixo da média.

Por exemplo, em uma distribuição onde a média é 6.000 e o desvio padrão é 1.000, o valor 8.000 possui Z-score 2. Isso pois ele está 2 desvios padrão acima da média.

Já o valor 5.000 possui Z-score -1, pois ele está 1 desvio padrão abaixo da média.

Pegou a ideia? Sua fórmula é bem simples:

$$
\text{Z-score} = \frac{\text{Valor} - \text{Média}}{\text{Desvio padrão}}
$$

Dada sua fórmula, ele possui uma característica muito interessante: a soma de todos os Z-scores de um conjunto de dados é igual a 0.

> 💡 Isso se dá pela 1ª Propriedade da Média Aritmética, que diz que a soma dos desvios de cada valor em relação à média aritmética de um conjunto de dados é sempre igual a 0.

> 🧠 Não sabia disso? Bom, agora você sabe um dos motivos pelos quais elevamos os desvios ao quadrado para calcular o desvio padrão.

Se você já conhecia o Z-score e essa propriedade, um ponto que talvez você não sabia é que a média de um subconjunto de valores pode ser encontrada da seguinte forma:

$$
\text{Média do subconjunto} = \text{Média do conjunto} \cdot \text{Desvio padrão do conjunto} \cdot \text{Z-score médio do subconjunto}
$$

Onde o Z-score médio do subconjunto é igual a:

$$
\text{Z-score médio do subconjunto} = \frac{1}{\text{n}} \cdot \sum_{i=1}^{n}{\text{Z-score}_{i}}
\\~\\
\text{n} = \text{Quantidade de elementos do subconjunto}
$$

Essa propriedade é muito importante para a solução que apresentarei, e é demonstrada abaixo neste Notebook, no capítulo: [Encontrando a média de cada subconjunto com Z-score](#capitulo1).

> 💡 Note que estamos trazendo uma solução bem mais complexa e, ao mesmo tempo, mais plausível, do que simplesmente calcular o fator utilizando o Ticket Médio de cada canal.

# A solução

Se o Z-score médio pode ser utilizado para calcular a média de um subconjunto, ele também pode ser utilizado para ajustar a meta para cada subconjunto.

Contudo, calcular o Z-score médio para cada subconjunto cai na mesma problemática que levantei acima, principalmente de performance no Power BI.

Além, onde eu poderia calcular esse Z-score? Nos valores atuais? Isso não seria justo, pois a meta foi definida antes desses valores sequer serem gerados.

Assim, a solução encontrada foi levar alguns dados prontos para o Power BI, e realizar os demais cálculos lá, da seguinte forma:

1. Dados prontos:

    1. Calcular a soma dos Z-scores e a quantidade de elementos de cada combinação das três colunas através de dados históricos, por exemplo, do último mês:

        | Tipo   | Canal      | Direcionamento   | Soma Z-scores | Quantidade |
        |--------|------------|------------------|---------------|------------|
        | Ads    | Facebook   | Blog             | 100.000       | 75.000     |
        | E-mail | Campanha   | Site             | - 300.000     | 125.000    |

        <br />
    1. Calcular o Coeficiente de Variação dos dados históricos:

        $$
        \text{Coeficiente de Variação} = \frac{\text{Desvio padrão}}{\text{Média}}
        $$
    
    1. Calcular a variação da meta:

        $$
        \text{Variação da Meta} = \text{Meta} \cdot \text{Coeficiente de Variação}
        $$

1. Power BI:

    1. Ajustar a meta do Ticket Médio para cada canal:

        $$
        \text{Meta ajustada} = \text{Meta} + \text{Variação da Meta} \cdot \frac{\sum_{i=1}^{n}{\text{Soma Z-score}_{i}}}{\sum_{i=1}^{n}{\text{Quantidade}_{i}}}
        $$

        <br />

        Em DAX:

        ```language-dax
        meta_ajustada = DISTINCT(fMeta[meta]) + 
                        DISTINCT(fMeta[variacao]) *
                        DIVIDE(
                            SUM(fBaseHistorica[soma_z_score],
                            SUM(fBaseHistorica[quantidade]),
                            0))
        ```
        
        > 🧠 Note que a soma dos Z-scores dividido pela soma das quantidades nada mais é do que o Z-score médio do subconjunto.

Como ao invés da tabela inteira levamos uma tabela sumarizada para o Power BI, reduzimos drasticamente o número de linhas dela.

Abaixo, no exemplo deixado neste Notebook, você verá que uma sumarização parecida reduz uma tabela de 5 milhões de linhas para menos de 18 mil.

> 🎉 Uma redução de 99,6%!

Além disso, os cálculos deixados para o Power BI são executados rapidamente utilizando o contexto de filtro.

# Prática

Agora eu te convido a executar esse Notebook e acompanhar, através dos exemplos com dados aleatórios, o que foi feito e como a meta foi ajustada para cada canal.

Além, tente mudar os parâmetros da função base_aleatoria e analisar os resultados gerados.

Espero que você tenha aprendido algo comigo, e que possa levar esse aprendizado para seu futuro!

<a id='capitulo1'></a>
# Encontrando a média de cada subconjunto com Z-score

In [1]:
# Importa as bibliotecas
import pandas as pd
import numpy as np
import string
import time
from typing import List

In [2]:
# Define uma função para criar uma base aleatória com valores outliers
def base_aleatoria(tamanho: int = 5_000_000,
                   percentual_outliers_baixo: float = 0.05,
                   percentual_outliers_alto: float = 0.05,
                   classes_outliers_baixo: List[str] = [['Y1', 'Y2', 'Y3'],
                                                        ['Z1', 'Z2', 'Z3']],
                   classes_outliers_alto: List[str] = [['A1', 'A2', 'A3'],
                                                       ['B1', 'B2', 'B3'],
                                                       ['C1', 'C2', 'C3'],
                                                       ['D1', 'D2', 'D3']],
                   media_normal: float = 6_000,
                   desvio_padrao_normal: float = 500,
                   media_outlier_baixo: int = 2_000,
                   desvio_padrao_outlier_baixo: int = 100,
                   media_outlier_alto: int = 10_000,
                   desvio_padrao_outlier_alto: int = 2_000) -> pd.DataFrame:
    
    horario_inicio = time.time()
    letras = list(string.ascii_uppercase)
    
    quantidade_outliers_baixo = int(tamanho * percentual_outliers_baixo)
    quantidade_outliers_alto = int(tamanho * percentual_outliers_alto)
    tamanho_sem_outliers = tamanho - quantidade_outliers_baixo - quantidade_outliers_alto
    quantidade_outliers_baixo_por_classe = quantidade_outliers_baixo // len(classes_outliers_baixo)
    quantidade_outliers_alto_por_classe = quantidade_outliers_alto // len(classes_outliers_alto)

    print(f'Criando os primeiros {tamanho_sem_outliers:_.0f} valores'.replace('.', ',').replace('_', '.'))
    classe_1 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '1')
    classe_2 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '2')
    classe_3 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '3')
    valores = np.around(np.random.normal(media_normal, desvio_padrao_normal, tamanho_sem_outliers), 2)
    horario_fim_1 = time.time()
    print(f'Valores criados em {(horario_fim_1 - horario_inicio):_.2f} segundos'.replace('.', ',').replace('_', '.'))

    for i, cl in enumerate(classes_outliers_baixo, 1):
        print(f'\nCriando os {quantidade_outliers_baixo_por_classe:_.0f} outliers com valores baixos do {i}º conjunto de classes'.replace('.', ',').replace('_', '.'))
        classe_1 = np.concatenate((classe_1, np.repeat(cl[0], quantidade_outliers_baixo_por_classe)), axis=None)
        classe_2 = np.concatenate((classe_2, np.repeat(cl[1], quantidade_outliers_baixo_por_classe)), axis=None)
        classe_3 = np.concatenate((classe_3, np.repeat(cl[2], quantidade_outliers_baixo_por_classe)), axis=None)
        valores = np.concatenate((valores, np.around(np.random.normal(media_outlier_baixo, desvio_padrao_outlier_baixo, quantidade_outliers_baixo_por_classe), 2)), axis=None)
        horario_fim_2 = time.time()
        print(f'Valores criados em {(horario_fim_2 - horario_fim_1):_.2f} segundos'.replace('.', ',').replace('_', '.'))
        horario_fim_1 = horario_fim_2

    for i, cl in enumerate(classes_outliers_alto, 1):
        print(f'\nCriando os {quantidade_outliers_alto_por_classe:_.0f} outliers com valores altos do {i}º conjunto de classes'.replace('.', ',').replace('_', '.'))
        classe_1 = np.concatenate((classe_1, np.repeat(cl[0], quantidade_outliers_alto_por_classe)), axis=None)
        classe_2 = np.concatenate((classe_2, np.repeat(cl[1], quantidade_outliers_alto_por_classe)), axis=None)
        classe_3 = np.concatenate((classe_3, np.repeat(cl[2], quantidade_outliers_alto_por_classe)), axis=None)
        valores = np.concatenate((valores, np.around(np.random.normal(media_outlier_alto, desvio_padrao_outlier_alto, quantidade_outliers_alto_por_classe), 2)), axis=None)
        horario_fim_2 = time.time()
        print(f'Valores criados em {(horario_fim_2 - horario_fim_1):_.2f} segundos'.replace('.', ',').replace('_', '.'))
        horario_fim_1 = horario_fim_2

    base = pd.DataFrame(dict(classe_1=classe_1, classe_2=classe_2, classe_3=classe_3, valor=valores))
    print(f'\nBase criada em {(horario_fim_2 - horario_inicio):.2f} segundos'.replace('.', ',').replace('_', '.'))
    return base

In [3]:
# Gera a base histórica aleatória
base_historica = base_aleatoria()

# Obtém algumas estatísticas da base
media_base_historica = base_historica.valor.mean()
desvio_padrao_base_historica = base_historica.valor.std()
coeficiente_variacao_base_historica = desvio_padrao_base_historica / media_base_historica

# Calcula o Z-score de cada valor
base_historica['z_score'] = (base_historica.valor - media_base_historica) / desvio_padrao_base_historica
display(base_historica)

Criando os primeiros 4.500.000 valores
Valores criados em 9,01 segundos

Criando os 125.000 outliers com valores baixos do 1º conjunto de classes
Valores criados em 0,10 segundos

Criando os 125.000 outliers com valores baixos do 2º conjunto de classes
Valores criados em 0,12 segundos

Criando os 62.500 outliers com valores altos do 1º conjunto de classes
Valores criados em 0,12 segundos

Criando os 62.500 outliers com valores altos do 2º conjunto de classes
Valores criados em 0,13 segundos

Criando os 62.500 outliers com valores altos do 3º conjunto de classes
Valores criados em 0,13 segundos

Criando os 62.500 outliers com valores altos do 4º conjunto de classes
Valores criados em 0,13 segundos

Base criada em 9,73 segundos


,classe_1,classe_2,classe_3,valor,z_score
0,V1,H2,J3,5082.86,-0.644372
1,B1,C2,E3,6671.14,0.471492
2,K1,X2,R3,6483.18,0.339438
3,P1,F2,L3,5890.74,-0.076787
4,W1,B2,W3,5365.13,-0.446060
...,...,...,...,...,...
4999995,D1,D2,D3,11771.03,4.054478
4999996,D1,D2,D3,9765.05,2.645154
4999997,D1,D2,D3,7668.01,1.171854
4999998,D1,D2,D3,10621.29,3.246715


In [4]:
# Mostra a média geral do Valor e a soma do Z-score (a soma tem que ser zero)
print(f'Média do Valor: {media_base_historica:_.2f}'.replace('.', ',').replace('_', '.'))
print(f'Soma do Z-score: {base_historica.z_score.sum():_.2f}'.replace('.', ',').replace('_', '.'))

Média do Valor: 6.000,04
Soma do Z-score: -0,00


In [5]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1
analise_classe_1 = base_historica.groupby('classe_1').agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1.reset_index(inplace=True)
analise_classe_1['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1.soma_z_score / analise_classe_1.quantidade)
analise_classe_1.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_classe_1.soma_z_score.sum()
soma_quantidade = analise_classe_1.quantidade.sum()
analise_classe_1 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1],
    ignore_index=True)

analise_classe_1['medias_iguais'] = analise_classe_1.media_real.round(2) == analise_classe_1.media_calculada_com_z_score.round(2)
display(analise_classe_1.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,B1,"175.787,72",235.288,"7.063,45","7.063,45",True
2,D1,"175.768,56",235.706,"7.061,45","7.061,45",True
3,C1,"175.743,89",236.153,"7.059,30","7.059,30",True
4,A1,"175.535,64",236.578,"7.056,14","7.056,14",True
5,E1,"203,40",172.847,"6.001,71","6.001,71",True
6,Q1,"201,96",172.571,"6.001,70","6.001,70",True
7,F1,"158,99",173.269,"6.001,34","6.001,34",True
8,J1,"128,68",173.141,"6.001,09","6.001,09",True
9,I1,"108,62",173.244,"6.000,93","6.000,93",True


In [6]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_2
analise_classe_2 = base_historica.groupby('classe_2').agg({'z_score': 'sum', 'classe_2': 'count', 'valor': 'mean'})
analise_classe_2.rename(columns={'z_score': 'soma_z_score', 'classe_2': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_2.reset_index(inplace=True)
analise_classe_2['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2.soma_z_score / analise_classe_2.quantidade)
analise_classe_2.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_classe_2.soma_z_score.sum()
soma_quantidade = analise_classe_2.quantidade.sum()
analise_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_2],
    ignore_index=True)

analise_classe_2['medias_iguais'] = analise_classe_2.media_real.round(2) == analise_classe_2.media_calculada_com_z_score.round(2)
display(analise_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_2,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,B2,"175.702,26",235.440,"7.062,25","7.062,25",True
2,A2,"175.612,05",235.333,"7.062,19","7.062,19",True
3,D2,"175.727,55",235.658,"7.061,42","7.061,42",True
4,C2,"175.795,79",235.882,"7.060,83","7.060,83",True
5,K2,"261,79",173.332,"6.002,19","6.002,19",True
6,G2,"161,54",172.770,"6.001,37","6.001,37",True
7,E2,"143,10",173.121,"6.001,21","6.001,21",True
8,T2,"142,74",172.974,"6.001,21","6.001,21",True
9,V2,"96,00",172.692,"6.000,83","6.000,83",True


In [7]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_3
analise_classe_3 = base_historica.groupby('classe_3').agg({'z_score': 'sum', 'classe_3': 'count', 'valor': 'mean'})
analise_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_3': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_3.reset_index(inplace=True)
analise_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_3.soma_z_score / analise_classe_3.quantidade)
analise_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_3.quantidade.sum()
analise_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_3=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_3],
    ignore_index=True)

analise_classe_3['medias_iguais'] = analise_classe_3.media_real.round(2) == analise_classe_3.media_calculada_com_z_score.round(2)
display(analise_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_3,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,D3,"175.925,88",235.526,"7.063,22","7.063,22",True
2,C3,"175.746,00",235.345,"7.062,95","7.062,95",True
3,A3,"175.567,71",235.521,"7.061,07","7.061,07",True
4,B3,"175.816,03",235.962,"7.060,59","7.060,59",True
5,L3,"506,97",172.396,"6.004,22","6.004,22",True
6,V3,"229,19",172.729,"6.001,92","6.001,92",True
7,W3,"144,79",172.608,"6.001,23","6.001,23",True
8,M3,"135,94",173.109,"6.001,15","6.001,15",True
9,F3,"60,80",172.902,"6.000,54","6.000,54",True


In [8]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1 + classe_2
analise_classe_1_classe_2 = base_historica.groupby(['classe_1', 'classe_2']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1_classe_2.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1_classe_2.reset_index(inplace=True)
analise_classe_1_classe_2['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade)
analise_classe_1_classe_2.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_classe_1_classe_2.soma_z_score.sum()
soma_quantidade = analise_classe_1_classe_2.quantidade.sum()
analise_classe_1_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1_classe_2],
    ignore_index=True)

analise_classe_1_classe_2['medias_iguais'] = analise_classe_1_classe_2.media_real.round(2) == analise_classe_1_classe_2.media_calculada_com_z_score.round(2)
display(analise_classe_1_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,classe_2,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,B1,B2,"175.851,05",68.980,"9.628,62","9.628,62",True
2,D1,D2,"175.609,51",69.155,"9.614,47","9.614,47",True
3,C1,C2,"175.796,74",69.270,"9.612,32","9.612,32",True
4,A1,A2,"175.611,10",69.216,"9.611,32","9.611,32",True
...,...,...,...,...,...,...,...
672,W1,B2,"-66,71",6.715,"5.985,90","5.985,90",True
673,O1,F2,"-80,21",6.707,"5.983,01","5.983,01",True
674,S1,X2,"-98,14",6.616,"5.978,92","5.978,92",True
675,Y1,Y2,"-351.315,32",131.707,"2.203,36","2.203,36",True


In [9]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1 + classe_3
analise_classe_1_classe_3 = base_historica.groupby(['classe_1', 'classe_3']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1_classe_3.reset_index(inplace=True)
analise_classe_1_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade)
analise_classe_1_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_classe_1_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_1_classe_3.quantidade.sum()
analise_classe_1_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1_classe_3],
    ignore_index=True)

analise_classe_1_classe_3['medias_iguais'] = analise_classe_1_classe_3.media_real.round(2) == analise_classe_1_classe_3.media_calculada_com_z_score.round(2)
display(analise_classe_1_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,classe_3,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,B1,B3,"175.874,41",69.175,"9.618,87","9.618,87",True
2,C1,C3,"175.739,83",69.144,"9.617,73","9.617,73",True
3,D1,D3,"175.608,69",69.096,"9.617,54","9.617,54",True
4,A1,A3,"175.578,70",69.306,"9.605,96","9.605,96",True
...,...,...,...,...,...,...,...
672,X1,D3,"-70,07",6.536,"5.984,78","5.984,78",True
673,T1,T3,"-73,75",6.606,"5.984,14","5.984,14",True
674,Z1,P3,"-82,47",6.687,"5.982,48","5.982,48",True
675,Z1,Z3,"-351.186,66",131.663,"2.203,48","2.203,48",True


In [10]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_2 + classe_3
analise_classe_2_classe_3 = base_historica.groupby(['classe_2', 'classe_3']).agg({'z_score': 'sum', 'classe_2': 'count', 'valor': 'mean'})
analise_classe_2_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_2': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_2_classe_3.reset_index(inplace=True)
analise_classe_2_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade)
analise_classe_2_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_classe_2_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_2_classe_3.quantidade.sum()
analise_classe_2_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_2_classe_3],
    ignore_index=True)

analise_classe_2_classe_3['medias_iguais'] = analise_classe_2_classe_3.media_real.round(2) == analise_classe_2_classe_3.media_calculada_com_z_score.round(2)
display(analise_classe_2_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_2,classe_3,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,C2,C3,"175.791,27",69.160,"9.617,95","9.617,95",True
2,B2,B3,"175.810,92",69.268,"9.612,71","9.612,71",True
3,D2,D3,"175.601,62",69.248,"9.609,45","9.609,45",True
4,A2,A3,"175.578,58",69.250,"9.608,87","9.608,87",True
...,...,...,...,...,...,...,...
672,O2,R3,"-80,71",6.694,"5.982,88","5.982,88",True
673,B2,V3,"-84,05",6.567,"5.981,82","5.981,82",True
674,X2,X3,"-91,03",6.667,"5.980,60","5.980,60",True
675,Z2,Z3,"-351.235,66",131.790,"2.206,61","2.206,61",True


In [11]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento de todas as classes
analise_geral = base_historica.groupby(['classe_1', 'classe_2', 'classe_3']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_geral.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_geral.reset_index(inplace=True)
analise_geral['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_geral.soma_z_score / analise_geral.quantidade)
analise_geral.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

soma_z_score = analise_geral.soma_z_score.sum()
soma_quantidade = analise_geral.quantidade.sum()
analise_geral = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_geral],
    ignore_index=True)

analise_geral['medias_iguais'] = analise_geral.media_real.round(2) == analise_geral.media_calculada_com_z_score.round(2)
display(analise_geral.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                    f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,classe_2,classe_3,soma_z_score,quantidade,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,,"-0,00",5.000.000,"6.000,04","6.000,04",True
1,B1,B2,B3,"175.873,67",62.740,"9.990,03","9.990,03",True
2,C1,C2,C3,"175.794,08",62.735,"9.988,54","9.988,54",True
3,A1,A2,A3,"175.601,09",62.757,"9.982,76","9.982,76",True
4,D1,D2,D3,"175.564,44",62.765,"9.981,43","9.981,43",True
...,...,...,...,...,...,...,...,...
17572,R1,V2,F3,"-22,45",269,"5.881,27","5.881,27",True
17573,L1,D2,V3,"-21,43",252,"5.878,97","5.878,97",True
17574,F1,K2,C3,"-21,62",239,"5.871,30","5.871,30",True
17575,Y1,Y2,Y3,"-351.257,44",125.285,"2.009,40","2.009,40",True


<a id='capitulo2'></a>
# Reajustando a meta com uma nova base

In [12]:
# Cria uma nova base para reajustar a meta
base = base_aleatoria(tamanho=1_000_000,
                      media_normal=7_000)

# Define um valor para a meta e calcula a variação da meta
meta = 7_000
variacao_meta = meta * coeficiente_variacao_base_historica

# Imprime a base na tela
display(base)

Criando os primeiros 900.000 valores
Valores criados em 1,81 segundos

Criando os 25.000 outliers com valores baixos do 1º conjunto de classes
Valores criados em 0,02 segundos

Criando os 25.000 outliers com valores baixos do 2º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 1º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 2º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 3º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 4º conjunto de classes
Valores criados em 0,02 segundos

Base criada em 1,93 segundos


,classe_1,classe_2,classe_3,valor
0,X1,X2,C3,6781.23
1,A1,E2,O3,6907.02
2,P1,W2,P3,7650.25
3,G1,K2,L3,7282.42
4,A1,V2,K3,6726.23
...,...,...,...,...
999995,D1,D2,D3,8447.08
999996,D1,D2,D3,10895.02
999997,D1,D2,D3,8342.65
999998,D1,D2,D3,13416.82


In [13]:
# Calcula a média geral do Valor
media_base = base.valor.mean()
print(f'Média do Valor: {media_base:_.2f}'.replace('.', ',').replace('_', '.'))

Média do Valor: 6.899,64


In [14]:
# Resume a base e a base histórica para juntá-las, visando facilitar os cálculos
base = base.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).valor.agg(valor='sum', quantidade='count')
base_historica_resumida = base_historica.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).z_score.agg(soma_z_score='sum', quantidade_historica='count')
base = base.merge(base_historica_resumida, how='outer', on=['classe_1', 'classe_2', 'classe_3']).fillna(0)
base = base.astype({'valor': float, 'quantidade': int, 'soma_z_score': float, 'quantidade_historica': int})
display(base)

,classe_1,classe_2,classe_3,valor,quantidade,soma_z_score,quantidade_historica
0,A1,A2,A3,1.253536e+08,12550,175601.091050,62757
1,A1,A2,B3,3.689341e+05,53,18.256547,249
2,A1,A2,C3,3.903684e+05,55,5.270987,262
3,A1,A2,D3,3.274048e+05,46,-5.422833,250
4,A1,A2,E3,4.022207e+05,57,1.693615,267
...,...,...,...,...,...,...,...
17571,Z1,Z2,V3,3.407613e+05,49,4.368749,254
17572,Z1,Z2,W3,3.831239e+05,55,-14.035538,268
17573,Z1,Z2,X3,3.878028e+05,55,4.345171,254
17574,Z1,Z2,Y3,3.011057e+05,43,-0.565638,253


In [15]:
# Calcula os desvios da classe_1 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1 = base.groupby('classe_1', as_index=False).sum(numeric_only=True)
analise_classe_1['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1.soma_z_score / analise_classe_1.quantidade_historica)
analise_classe_1['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1.soma_z_score / analise_classe_1.quantidade_historica)
analise_classe_1['valor_medio'] = base.valor / base.quantidade
analise_classe_1['desvio_percentual'] = (analise_classe_1.valor_medio / analise_classe_1.meta_ajustada_com_z_score) - 1
analise_classe_1.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_classe_1.valor.sum()
soma_quantidade = analise_classe_1.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_classe_1 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_classe_1],
    ignore_index=True)

analise_classe_1.desvio_percentual = analise_classe_1.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,Y1,"293.404.199,01",59.779,"-351.280,70",297.828,"4.321,21","5.041,39","7.046,95","39,78%"
2,Z1,"291.252.836,99",59.458,"-351.479,39",297.913,"4.320,74","5.040,84","6.982,55","38,52%"
3,A1,"367.407.633,11",47.111,"175.535,64",236.578,"7.056,14","8.232,11","9.988,34","21,33%"
4,F1,"239.622.691,88",34.234,"158,99",173.269,"6.001,34","7.001,52","7.127,13","1,79%"
5,L1,"241.086.335,01",34.438,"-23,42",172.126,"5.999,84","6.999,77","7.108,62","1,55%"
6,T1,"242.041.352,72",34.604,"-185,17",173.473,"5.998,52","6.998,23","7.096,70","1,41%"
7,J1,"243.442.340,40",34.787,"128,68",173.141,"6.001,09","7.001,23","7.099,64","1,41%"
8,E1,"242.144.760,02",34.584,"203,40",172.847,"6.001,71","7.001,95","7.056,50","0,78%"
9,V1,"240.445.401,91",34.362,"-161,48",173.191,"5.998,71","6.998,45","7.043,23","0,64%"


In [16]:
# Calcula os desvios da classe_2 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_2 = base.groupby('classe_2', as_index=False).sum(numeric_only=True)
analise_classe_2['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2.soma_z_score / analise_classe_2.quantidade_historica)
analise_classe_2['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_2.soma_z_score / analise_classe_2.quantidade_historica)
analise_classe_2['valor_medio'] = base.valor / base.quantidade
analise_classe_2['desvio_percentual'] = (analise_classe_2.valor_medio / analise_classe_2.meta_ajustada_com_z_score) - 1
analise_classe_2.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_classe_2.valor.sum()
soma_quantidade = analise_classe_2.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_2.soma_z_score.sum()
soma_quantidade_historica = analise_classe_2.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_classe_2],
    ignore_index=True)

analise_classe_2.desvio_percentual = analise_classe_2.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_2,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,Y2,"292.188.819,15",59.620,"-351.541,87",298.527,"4.323,90","5.044,52","7.046,95","39,70%"
2,Z2,"291.841.630,31",59.565,"-351.278,40",297.625,"4.320,08","5.040,06","6.982,55","38,54%"
3,A2,"366.610.124,03",47.003,"175.612,05",235.333,"7.062,19","8.239,17","9.988,34","21,23%"
4,F2,"242.066.043,15",34.570,"-126,63",173.090,"5.998,99","6.998,79","7.127,13","1,83%"
5,L2,"242.247.047,99",34.613,"-41,80",172.930,"5.999,69","6.999,60","7.108,62","1,56%"
6,J2,"239.366.531,50",34.211,"74,86",173.253,"6.000,65","7.000,72","7.099,64","1,41%"
7,T2,"242.795.022,04",34.674,"142,74",172.974,"6.001,21","7.001,37","7.096,70","1,36%"
8,E2,"241.689.607,99",34.531,"143,10",173.121,"6.001,21","7.001,37","7.056,50","0,79%"
9,V2,"241.165.017,39",34.440,"96,00",172.692,"6.000,83","7.000,92","7.043,23","0,60%"


In [17]:
# Calcula os desvios da classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_3 = base.groupby('classe_3', as_index=False).sum(numeric_only=True)
analise_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_3.soma_z_score / analise_classe_3.quantidade_historica)
analise_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_3.soma_z_score / analise_classe_3.quantidade_historica)
analise_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_3['desvio_percentual'] = (analise_classe_3.valor_medio / analise_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_3.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_classe_3.valor.sum()
soma_quantidade = analise_classe_3.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_3.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_3=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_classe_3],
    ignore_index=True)

analise_classe_3.desvio_percentual = analise_classe_3.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_3,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,Y3,"294.342.233,78",59.923,"-351.278,41",297.844,"4.321,32","5.041,50","7.046,95","39,78%"
2,Z3,"294.605.612,17",59.947,"-351.287,25",298.331,"4.324,01","5.044,65","6.982,55","38,41%"
3,A3,"367.854.239,53",47.191,"175.567,71",235.521,"7.061,07","8.237,87","9.988,34","21,25%"
4,F3,"243.415.474,80",34.780,"60,80",172.902,"6.000,54","7.000,58","7.127,13","1,81%"
5,L3,"242.150.108,51",34.598,"506,97",172.396,"6.004,22","7.004,88","7.108,62","1,48%"
6,J3,"240.512.657,29",34.340,"-306,94",173.319,"5.997,52","6.997,06","7.099,64","1,47%"
7,T3,"242.688.838,24",34.690,"-174,85",173.022,"5.998,60","6.998,32","7.096,70","1,41%"
8,E3,"242.303.528,67",34.609,"-26,89",173.440,"5.999,82","6.999,74","7.056,50","0,81%"
9,V3,"240.937.673,73",34.436,"229,19",172.729,"6.001,92","7.002,20","7.043,23","0,59%"


In [18]:
# Calcula os desvios da classe_1 + classe_2 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1_classe_2 = base.groupby(['classe_1', 'classe_2'], as_index=False).sum(numeric_only=True)
analise_classe_1_classe_2['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade_historica)
analise_classe_1_classe_2['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade_historica)
analise_classe_1_classe_2['valor_medio'] = base.valor / base.quantidade
analise_classe_1_classe_2['desvio_percentual'] = (analise_classe_1_classe_2.valor_medio / analise_classe_1_classe_2.meta_ajustada_com_z_score) - 1
analise_classe_1_classe_2.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_classe_1_classe_2.valor.sum()
soma_quantidade = analise_classe_1_classe_2.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1_classe_2.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1_classe_2.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_classe_1_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_classe_1_classe_2],
    ignore_index=True)

analise_classe_1_classe_2.desvio_percentual = analise_classe_1_classe_2.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,classe_2,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,Y1,Y2,"59.647.535,79",26.382,"-351.315,32",131.707,"2.203,36","2.570,57","7.008,64","172,65%"
2,Z1,Z2,"59.306.873,70",26.335,"-351.250,67",131.545,"2.199,38","2.565,93","6.947,77","170,77%"
3,M1,H2,"9.312.223,96",1.332,"-46,59",6.553,"5.989,92","6.988,19","7.199,34","3,02%"
4,R1,N2,"8.899.803,20",1.273,"22,76",6.693,"6.004,88","7.005,65","7.194,03","2,69%"
...,...,...,...,...,...,...,...,...,...,...
672,C1,G2,"9.406.600,70",1.342,"12,54",6.710,"6.002,70","7.003,10","6.760,66","-3,46%"
673,A1,A2,"134.582.466,93",13.869,"175.611,10",69.216,"9.611,32","11.213,13","9.988,34","-10,92%"
674,D1,D2,"134.048.546,91",13.808,"175.609,51",69.155,"9.614,47","11.216,81","6.993,74","-37,65%"
675,C1,C2,"134.094.589,63",13.817,"175.796,74",69.270,"9.612,32","11.214,30","6.976,58","-37,79%"


In [19]:
# Calcula os desvios da classe_1 + classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1_classe_3 = base.groupby(['classe_1', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_classe_1_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade_historica)
analise_classe_1_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade_historica)
analise_classe_1_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_1_classe_3['desvio_percentual'] = (analise_classe_1_classe_3.valor_medio / analise_classe_1_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_1_classe_3.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_classe_1_classe_3.valor.sum()
soma_quantidade = analise_classe_1_classe_3.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1_classe_3.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_classe_1_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_classe_1_classe_3],
    ignore_index=True)

analise_classe_1_classe_3.desvio_percentual = analise_classe_1_classe_3.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,classe_3,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,Y1,Y3,"59.424.296,56",26.344,"-351.246,75",131.661,"2.202,77","2.569,88","7.008,64","172,72%"
2,Z1,Z3,"59.289.041,61",26.329,"-351.186,66",131.663,"2.203,48","2.570,71","6.947,77","170,27%"
3,R1,N3,"9.314.243,90",1.329,"28,50",6.518,"6.006,26","7.007,26","7.194,03","2,67%"
4,M1,H3,"9.129.643,07",1.304,"57,58",6.668,"6.012,33","7.014,34","7.199,34","2,64%"
...,...,...,...,...,...,...,...,...,...,...
672,C1,G3,"9.272.410,22",1.323,"2,87",6.742,"6.000,64","7.000,71","6.760,66","-3,43%"
673,A1,A3,"134.586.007,07",13.868,"175.578,70",69.306,"9.605,96","11.206,89","9.988,34","-10,87%"
674,D1,D3,"133.978.733,96",13.796,"175.608,69",69.096,"9.617,54","11.220,39","6.993,74","-37,67%"
675,C1,C3,"134.367.681,32",13.856,"175.739,83",69.144,"9.617,73","11.220,61","6.976,58","-37,82%"


In [20]:
# Calcula os desvios da classe_2 + classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_2_classe_3 = base.groupby(['classe_2', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_classe_2_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade_historica)
analise_classe_2_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade_historica)
analise_classe_2_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_2_classe_3['desvio_percentual'] = (analise_classe_2_classe_3.valor_medio / analise_classe_2_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_2_classe_3.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_classe_2_classe_3.valor.sum()
soma_quantidade = analise_classe_2_classe_3.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_2_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_2_classe_3.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_classe_2_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_classe_2_classe_3],
    ignore_index=True)

analise_classe_2_classe_3.desvio_percentual = analise_classe_2_classe_3.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_2_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_2,classe_3,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,Y2,Y3,"58.989.921,69",26.287,"-351.240,42",131.693,"2.203,76","2.571,04","7.008,64","172,60%"
2,Z2,Z3,"59.558.474,63",26.370,"-351.235,66",131.790,"2.206,61","2.574,36","6.947,77","169,88%"
3,M2,H3,"9.280.947,35",1.330,"-10,24",6.634,"5.997,84","6.997,44","7.199,34","2,89%"
4,R2,N3,"8.832.701,49",1.260,"39,36",6.647,"6.008,46","7.009,83","7.194,03","2,63%"
...,...,...,...,...,...,...,...,...,...,...
672,C2,G3,"9.112.358,49",1.306,"52,48",6.615,"6.011,33","7.013,17","6.760,66","-3,60%"
673,A2,A3,"134.761.766,20",13.892,"175.578,58",69.250,"9.608,87","11.210,29","9.988,34","-10,90%"
674,D2,D3,"133.914.982,49",13.788,"175.601,62",69.248,"9.609,45","11.210,96","6.993,74","-37,62%"
675,C2,C3,"134.013.671,63",13.806,"175.791,27",69.160,"9.617,95","11.220,87","6.976,58","-37,82%"


In [21]:
# Calcula os desvios para cada elemento de todas as classes em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_geral = base.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_geral['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_geral.soma_z_score / analise_geral.quantidade_historica)
analise_geral['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_geral.soma_z_score / analise_geral.quantidade_historica)
analise_geral['valor_medio'] = base.valor / base.quantidade
analise_geral['desvio_percentual'] = (analise_geral.valor_medio / analise_geral.meta_ajustada_com_z_score) - 1
analise_geral.sort_values(by='desvio_percentual', ascending=False, ignore_index=True, inplace=True)

soma_valor = analise_geral.valor.sum()
soma_quantidade = analise_geral.quantidade.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_geral.soma_z_score.sum()
soma_quantidade_historica = analise_geral.quantidade_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_percentual = (valor_medio / meta_ajustada_com_z_score) - 1

analise_geral = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_percentual=[desvio_percentual])),
    analise_geral],
    ignore_index=True)

analise_geral.desvio_percentual = analise_geral.desvio_percentual.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_geral.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                                f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

,classe_1,classe_2,classe_3,valor,quantidade,soma_z_score,quantidade_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_percentual
0,Total,,,"6.899.638.470,40",1.000.000,"-0,00",5.000.000,"6.000,04","7.000,00","6.899,64","-1,43%"
1,D1,W2,P3,"370.525,54",51,"-7,30",242,"5.957,10","6.949,91","7.265,21","4,54%"
2,L1,R2,N3,"276.948,57",38,"-2,55",261,"5.986,14","6.983,79","7.288,12","4,36%"
3,B1,A2,I3,"318.132,02",44,"-9,97",246,"5.942,33","6.932,68","7.230,27","4,29%"
4,N1,G2,E3,"333.559,11",46,"-4,76",248,"5.972,73","6.968,15","7.251,28","4,06%"
...,...,...,...,...,...,...,...,...,...,...,...
17572,A1,A2,A3,"125.353.620,15",12.550,"175.601,09",62.757,"9.982,76","11.646,49","9.988,34","-14,24%"
17573,Z1,Z2,Z3,"50.320.674,78",25.048,"-351.245,28",125.238,"2.008,04","2.342,70","2.008,97","-14,25%"
17574,D1,D2,D3,"125.266.046,59",12.551,"175.564,44",62.765,"9.981,43","11.644,93","9.980,56","-14,29%"
17575,B1,B2,B3,"125.268.038,83",12.550,"175.873,67",62.740,"9.990,03","11.654,96","9.981,52","-14,36%"


# Fim